In [4]:
import pandas as pd

In [11]:
df = pd.read_csv('mission_launches.csv') # create a data frame for the launches
df_copy = df.copy() # create a copy of the data frame to preserve the original data

In [12]:
df_copy.head()

,Unnamed: 0.1,Unnamed: 0,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [43]:
df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], inplace=True)
df.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


Launch Frequency:

What year had the most launches?

What is the percent change?

For the Falcon 9, is the launch tempo increasing?

In [59]:
date_testing = pd.to_datetime(df['Date'], errors='coerce')
date_testing.head()
df['Month'] = date_testing.dt.month.fillna(0).astype(int)
df['Year'] = date_testing.dt.year.fillna(0).astype(int)
df['Time'] = date_testing.dt.strftime('%H:%M').fillna('00:00')
df.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status,Month,Year,Time
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success,8,2020,05:12
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success,8,2020,04:01
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success,8,2020,23:57
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success,7,2020,21:25
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success,7,2020,11:50


In [98]:
# Split 'Location' column by comma
split_cols = df['Location'].str.split(',', expand=True)

# Assign split columns to new DataFrame
af = pd.DataFrame()
af['Organisation'] = df['Organisation'].astype('category')
af['Launch Pad'] = split_cols[0].str.strip().astype('category')
af['Launch Center'] = split_cols[1].str.strip().astype('category') if split_cols.shape[1] > 1 else None

# Determine Launch State and Launch Country
def determine_state_and_country(row):
    parts = row[['Launch State', 'Launch Country']].values
    # print(parts)
    # If the last part is not 'None' and there is no state, it should be the country
    if len(parts) > 1 and pd.isna(parts[1]):
        # print('No State')
        # print(parts[0])
        return pd.Series([pd.NA, parts[0]])
    elif len(parts) > 1:
        # print('State and Country')
        return pd.Series([parts[0], parts[1]])
    else:
        # print('No state/country combination')
        return pd.Series([pd.NA, pd.NA])

af['Launch State'] = split_cols[2].str.strip().astype('category') if split_cols.shape[1] > 2 else pd.NA
af['Launch Country'] = split_cols[3].str.strip().astype('category') if split_cols.shape[1] > 3 else pd.NA

# Correcting the values using the defined function
af[['Launch State', 'Launch Country']] = af.apply(determine_state_and_country, axis=1).astype('category')
af.head()

,Organisation,Launch Pad,Launch Center,Launch State,Launch Country
0,SpaceX,LC-39A,Kennedy Space Center,Florida,USA
1,CASC,Site 9401 (SLS-2),Jiuquan Satellite Launch Center,NaN,China
2,SpaceX,Pad A,Boca Chica,Texas,USA
3,Roscosmos,Site 200/39,Baikonur Cosmodrome,NaN,Kazakhstan
4,ULA,SLC-41,Cape Canaveral AFS,Florida,USA


In [96]:
af['Launch State'].value_counts()


Launch State
Florida             834
California          454
French Guiana       299
Virginia             31
Texas                13
Marshall Islands      9
Algeria               4
Alaska                3
Maranh?œo             3
Name: count, dtype: int64

In [97]:
af['Launch Country'].value_counts()


Launch Country
Russia                            1390
USA                               1344
Kazakhstan                         701
France                             303
China                              259
Japan                              119
India                               76
Pacific Ocean                       36
New Zealand                         13
Iran                                13
Israel                              11
Kenya                                9
Australia                            6
North Korea                          5
New Mexico                           4
Barents Sea                          3
Brazil                               3
South Korea                          3
Gran Canaria                         2
Pacific Missile Range Facility       1
Name: count, dtype: int64

Questions:

What organization is the most successful?

What rocket is the most successful?

Which launch pad is the most used?

What month of the year has the most launches per year and on average?

In [24]:
df['Rocket_Status'] = df['Rocket_Status'].astype('category')

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   Organisation    4324 non-null   object  
 1   Location        4324 non-null   object  
 2   Date            4324 non-null   object  
 3   Detail          4324 non-null   object  
 4   Rocket_Status   4324 non-null   category
 5   Price           964 non-null    object  
 6   Mission_Status  4324 non-null   object  
dtypes: category(1), object(6)
memory usage: 207.2+ KB


In [29]:
df['Rocket_Status'].value_counts()

Rocket_Status
StatusRetired    3534
StatusActive      790
Name: count, dtype: int64

In [31]:
is_retired_filter = df['Rocket_Status'] == 'StatusRetired'
is_active_filter = df['Rocket_Status'] == 'StatusActive'

is_retired = df[is_retired_filter]
is_active = df[is_active_filter]

is_retired['Date'].info()

<class 'pandas.core.series.Series'>
Index: 3534 entries, 29 to 4323
Series name: Date
Non-Null Count  Dtype 
--------------  ----- 
3534 non-null   object
dtypes: object(1)
memory usage: 55.2+ KB


In [58]:
# df['Year'].value_counts()
df['Year'].dropna().describe()

TypeError: Series.dropna() takes 1 positional argument but 2 were given